By default, CrewAI uses OpenAI models (like GPT) for tasks if you don't explicitly provide a custom LLM (Language Model) or set llm=None in your Agent or Task.

If you want to use dummy or custom functions (like the flight checks in your example), you need to explicitly disable GPT usage by setting llm=None in both the Agent and Task.

 1. CrewAI is a python framework designed to connect multiple AI agents to work on a complex task.

 2. Langchain is a pythom framework to build LLM application.

 3. LangChain = Text generation + NLP tasks.

    Steps: Retrieve → Process → Generate (ideal for chatbots or text-based apps).

 4. CrewAI = Multiple agents + Tasks.

    Steps: Define agents → Delegate tasks → Orchestrate (ideal for managing complex workflows like travel planning).

Goal: User Input: The user asks a question like, “I want to go to Japan next month, can you help me book my trip?”.

CrewAI Workflow: The system will delegate different tasks to agents, each performing their own role:

Agent 1 (Flight Agent) checks available flights.

Agent 2 (Hotel Agent) suggests hotels.

Agent 3 (Advice Agent) gives travel tips.

.............................................................................................................

1. OpenAI API (Cloud):

Use this for OpenAI's models (e.g., GPT-3, GPT-4 can be used via API).

You authenticate with openai.api_key="  " and use openai.ChatCompletion.create(model=...) to interact with the model.

2. Local Model Server (e.g., using Ollama or similar):

Use this for open-source models (like LLaMA, Phi).

You connect to your local model server (e.g., http://localhost:11434/v1) using openai.OpenAI (base_url"" ,api_key"no key) and use .chat.completions.create(model=,messages=)

No API key may be needed, depending on your local setup.

In [26]:
import streamlit as st # webchatbot
from crewai import Agent, Task, Crew
from dotenv import load_dotenv #load secret keys (password or API) from  .env file
import os
import litellm #wrapper that lets use many models with the same code

In [27]:
load_dotenv() #accessed using os.getenv().
apikey=os.getenv("OPENAI_API_KEY")# get variable named openai api key from .env file

OpenAI's API only lets you use OpenAI models (like gpt-4, gpt-3.5-turbo).

LiteLLM is a python library acts as a wrapper that lets you use many models (OpenAI, Anthropic Claude, Google Gemini, Mistral, etc.) with the same code style. Switch to another model provider by just changing the model name

litellm.completion( model name="...",messages=[{'role':'','content':''}],api_key)....... choices[0].message.content

In [28]:
st.set_page_config(page_title="Travel Assistant",layout='centered')
st.title("Hello Iam your travel assistant")
destination=st.text_input("Enter you destination here")



2025-05-12 17:18:48.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.813 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.813 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.821 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.821 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-12 17:18:48.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Without llm=None: 

By default, CrewAI uses the LLM specified in the Agent or Task.

The Agent’s description is passed to the LLM when the task is executed.

The LLM processes this and provides a response, which is returned as the task output.

In [29]:
#Define Agents
#flight agent
flight_agent=Agent(
    role='Reservation Specialist',
    goal='Find suitable airlines ',
    backstory='Search for suitable airlines with special prices for the user',
    llm=None #llm used to generate answer
)

Each agent is defined with:

Role → Who the agent is ("Travel Expert", "Market Analyst", etc.)

Goal → What the agent is trying to accomplish

Backstory / Description → Optional extra context or personality

LLM → The language model the agent uses to think or act

Tasks → The actions the agent performs (calls to the LLM using the role, goal, etc.)

In [30]:
#Hotel agent
hotel_agent=Agent(
    role='Hotel Specialist',
    goal='Find suitable hotels',
    backstory='I specialize in recommending the best hotels and accommodations for travelers',
    llm=None #llm used to generate answer
)

In [31]:
#Tour agent
tour_agent=Agent(
    role='Tour Specialist',
    goal='Find suitable places to visit',
    backstory='I provide information on the best places to visit and special tours for tourists',
    llm=None #llm used to generate answer
)

In [32]:
#Advice agent
advice_agent=Agent(
    role='Advice Specialist',
    goal='Provide advices to the user',
    backstory='I give tips and important advice to help tourists have a smooth and enjoyable trip.',
    llm=None #llm used to generate answer
)

In [ ]:
#agent 1 will retreive information from a dummy function
def check_flights(destination):
    print(f"Fetching flight details for: {destination}")
    if destination.lower()=='canada':
        return [{'airline':'Emirates','price':'1800$'},{'airline':'Turkish','price':'1500$'}]
    elif destination.lower()=='japan':
        return [{'airline':'MiddleEast','price':'1500$'},{'airline':'AirFrance','price':'1400$'}]
    else:
        return[{'airline': 'None', 'price': 'N/A'}]


In [34]:
#rest agents will retrive their informations from openai model (ex. gpt-3.5-turbo) using litellm library
def check_hotels(destination):
    res=litellm.completion.create(
        model="gpt-3.5-turbo",
        api_key=apikey,
        temperature=0.3,
        messages=[{'role':'user','content':f"list 5 famous hotels in {destination}"}]
)
    return res['choices'][0]['message']['content']

print(type(res)) if its dic use res1['choices'][0]['message']['content']

                  if its object use  res.choices[0].message.content

In [35]:
def tours(destination):
    res=litellm.completion.create(
        model="gpt-3.5-turbo",
        api_key=apikey,
        temperature=0.3,
        messages=[{'role':'user','content':f"list 5 famous places to  be visited in {destination}"}]
)
    return res['choices'][0]['message']['content']

In [36]:
def advices(destination):
    res=litellm.completion.create(
        model="gpt-3.5-turbo",
        api_key=apikey,
        temperature=0.3,
        messages=[{'role':'user','content':f"list advices before traveling to {destination}"}]
)
    return res['choices'][0]['message']['content']

In general:

✅ Role, goal, and description are passed to the LLM when a task runs.

❌ Expected output is not passed unless you write it into the task instructions.



In [ ]:
#define tasks for each agent (description,agent,expected output...)
flight_task=Task(
    description="Find available airlines with their prices",
    expected_output="Provide a brief summary of the best flight option for the user",
    agent=flight_agent,
    llm=None,
    task_function=lambda: check_flights(destination)
)

In [ ]:
hotel_task=Task(
    description="Find famous hotels for the user",
    expected_output="Provide a brief summary of the best hotels for the user",
    agent=hotel_agent,
    llm=None,
    task_function=lambda: check_hotels(destination)
)

In [ ]:
tour_task=Task(
    description="Find famous places for the user",
    expected_output="Provide a brief summary of the best tour for the user",
    agent=tour_agent,
    llm=None,
    task_function=lambda: tours(destination)
)

In [ ]:
advice_task=Task(
    description="Provide advices for the user",
    expected_output="Give tips and important advice to help tourists have a smooth and enjoyable trip",
    agent=advice_agent,
    llm=None,
    task_function=lambda: advices(destination)
)

If you want to use your own custom function (that uses LiteLLM, openai, or anything else) instead of having CrewAI handle the LLM calls internally, then here's what to do:

✅ Steps to use a custom function with CrewAI:

Set the agent’s llm=None

Create a function_task

Pass your external function to the Task

In [41]:
crew=Crew(
    agents=[flight_agent,hotel_agent,tour_agent,advice_agent],
    tasks=[flight_task,hotel_task,tour_task,advice_task],
    process="sequential"
)

| Using Built-in LLM (`llm=OpenAI(...)`) | Using `function=` + your own logic    |
| -------------------------------------- | ------------------------------------- |
| CrewAI builds the full prompt          | You build the prompt manually         |
| Sends role, goal, backstory, etc.      | Only sends what your function defines |
| Easy to use, less flexible             | Fully flexible, more control          |


In [ ]:
if destination:
    result = crew.kickoff()
    #st.write("Results: ", result)# displays dict raws, output tasks....

    # Display chat history
    
    if "history" not in st.session_state:
        st.session_state.history = []

        #i want to skip printing expected output,description...

    for task_output in result.tasks_output:
        # Add each task response to the history
        st.session_state.history.append({"question": destination, "answer": task_output.raw})

    for chat in st.session_state.history:
        st.markdown(f"**You:** {chat['question']}")
        st.markdown(f"**Bot:** {chat['answer']}")
        st.markdown("----------")


CrewAI's task system just to:

Organize agents and tasks.

Run my custom functions in sequence.

Collect and display the output.

just type on cmd set OPENAI_API_KEY=put your api key here without "" before streamlit run name.py


Result :

"raw" — the top-level final response(last task executed in the crew).

"tasks_output" — all tasks results in order